In [6]:
from collections import defaultdict
def seven_segment_search1(file_name):
    f = open(file_name, 'r')
    digit1478_lens = set([2,3,4,7]) # 2: 1, 3: 7, 4: 4, 7: 8
    def find1478(arr):
        count = 0
        for bunch in arr.split():
            n = len(bunch)
            if n in digit1478_lens:
                count += 1
        return count
    total_count = 0
    for line in f:
        before_delim, after_delim = line.split(' | ')
        total_count += find1478(after_delim)
    return total_count
print(seven_segment_search1('input.txt'))

375


In [ ]:
from collections import defaultdict
def seven_segment_search2(file_name):
    f = open(file_name, 'r')
    eight_segs = set(['top', 'tl', 'tr', 'mid', 'bl', 'br', 'bot'])
    zero_segs = eight_segs - set(['mid'])
    two_segs = set(['top', 'tr', 'mid', 'bl', 'bot'])
    three_segs = set(['top', 'tr', 'mid', 'br', 'bot'])
    five_segs = set(['top', 'tl', 'mid', 'br', 'bot'])
    six_segs = eight_segs - set(['tr'])
    nine_segs = eight_segs - set(['bl'])
    def decode_before(arr):
        zero_p, one_p, two_p, three_p, four_p, five_p, six_p, seven_p, eight_p, nine_p = [set()]*10
        top, mid, bot, tl, tr, bl, br = [None]*7
        five_segs, six_segs = [], []
        char_to_freq = defaultdict(int) # top: 8, tl: 6*, tr: 8, mid: 7, bl: 4*, br: 9*, bot: 7
        freq_to_chs = defaultdict(set)
        for bunch in arr:
            n = len(bunch)
            bunch_set = set(bunch)
            if n == 2: one_p = bunch_set # 1
            elif n == 3: seven_p = bunch_set # 7
            elif n == 4: four_p = bunch_set # 4
            elif n == 5: five_segs.append(bunch_set) # 2, 3, 5
            elif n == 6: six_segs.append(bunch_set) # 0, 6, 9
            elif n == 7: eight_p = bunch_set # 8
            for ch in bunch: char_to_freq[ch] += 1
#         return segment_map # e.g. {a: bottom_left, ...}
        for ch, freq in char_to_freq.items(): freq_to_chs[freq].add(ch)
        # set difference
        top = list(seven_p - one_p)[0] # solve top by diff of 7 seg and 1 seg
        # solve tl, bl, br from unique freqs
        assert len(freq_to_chs[6]) == 1
        assert len(freq_to_chs[4]) == 1
        assert len(freq_to_chs[9]) == 1
        tl = list(freq_to_chs[6])[0]
        bl = list(freq_to_chs[4])[0]
        br = list(freq_to_chs[9])[0]
        # by deduction, solve tr = 1_p - br
        tr = list(one_p - set(br))[0]
        # rem is mid and bot
        assert len(list(four_p - set(tl) - set(tr) - set(br))) == 1
        mid = list(four_p - set(tl) - set(tr) - set(br))[0]
        assert len(freq_to_chs[7]) == 2
        for ch in freq_to_chs[7]: 
            if ch != mid: bot = ch
        print(f'{top=}, {tr=}, {br=}, {tl=}, {bl=}, {mid=}, {bot=}')
        assert len(set(top) | set(tr) | set(br) | set(tl) | set(bl) | set(mid)| set(bot)) == 7
        return {top: 'top', tr: 'tr', br: 'br', tl: 'tl', bl: 'bl', mid: 'mid', bot: 'bot'}
    def decode_after(arr, segment_map):
        nums_lst = []
#         print(f'{arr=}, {segment_map=}')
        for bunch in arr:
            n = len(bunch)
            bunch_set = set(bunch)
            if n == 2: nums_lst.append('1')
            elif n == 3: nums_lst.append('7')
            elif n == 4: nums_lst.append('4')
            elif n == 7: nums_lst.append('8')
            elif n == 5: 
                added = False
                for ch in bunch_set:
                    seg = segment_map[ch]
                    if seg == 'tl': nums_lst.append('5'); added = True; break
                    elif seg == 'bl': nums_lst.append('2'); added = True; break
                if not added: nums_lst.append('3')
            elif n == 6:
                segs_set = set()
                for ch in bunch_set:
                    segs_set.add(segment_map[ch])
                if 'mid' not in segs_set: nums_lst.append('0')
                elif 'tr' not in segs_set: nums_lst.append('6')
                elif 'bl' not in segs_set: nums_lst.append('9')
                print(f'{bunch=}, {segs_set=}, {nums_lst=}')

        return int(''.join(nums_lst))

    def decode_and_solve(arr):
        before_delim, after_delim = line.split(' | ')
        ch_to_seg = decode_before(before_delim.split())
        val = decode_after(after_delim.split(), ch_to_seg) 
        print(val)
        return val
    total_val = 0
    for line in f:
        total_val += decode_and_solve(line)
    return total_val
print(seven_segment_search2('input.txt'))